In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time

In [16]:
all_matches = []
main_url= "https://fbref.com/en/comps/106/2019/2019-Womens-World-Cup-Stats"

In [17]:
years= [2019, 2015]
years

[2019, 2015]

In [18]:
for year in years:
  time.sleep(10)
  data= requests.get(main_url)
  soup= BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[7] #css selector of the main table
  
  links = [l.get("href") for l in standings_table.find_all('a')] # get the links in the table
  links = [l for l in links if '/squads/' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

  
  previous_competition= soup.select("a.prev")[0].get("href")
  main_url = f"https://fbref.com{previous_competition}"
  

  for team_url in team_urls: # to get each team in the main table
    time.sleep(10)
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    html= StringIO(data.text)
    try:
        matches = pd.read_html(html, match="Scores & Fixtures")[0]
    except ValueError:
        continue
    time.sleep(3)
    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and '/shooting/' in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    html= StringIO(data.text)
    time.sleep(3)
    try:
        shooting = pd.read_html(html, match="Shooting")[0]
    except ValueError:
        continue
    shooting.columns = shooting.columns.droplevel()
    time.sleep(3)
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "SoT%", "G/Sh", "G/SoT", "Dist", "PK", "PKatt"]], on="Date")
    except ValueError:
        continue
    team_data = team_data[team_data["Comp"] == "World Cup"]

    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(3)



In [19]:
WWC_past_matches= pd.concat(all_matches)
WWC_past_matches

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Sh,SoT,SoT%,G/Sh,G/SoT,Dist,PK,PKatt,Season,Team
10,2019-06-11,21:00,World Cup,Group stage,Tue,Home,W,13,0,th Thailand,...,40.0,20.0,50.0,0.33,0.65,12.8,0.0,0.0,2019,United States Women
11,2019-06-16,18:00,World Cup,Group stage,Sun,Home,W,3,0,cl Chile,...,25.0,9.0,36.0,0.12,0.33,16.0,0.0,1.0,2019,United States Women
12,2019-06-20,21:00,World Cup,Group stage,Thu,Away,W,2,0,se Sweden,...,16.0,4.0,25.0,0.06,0.25,18.8,0.0,0.0,2019,United States Women
13,2019-06-24,18:00,World Cup,Round of 16,Mon,Away,W,2,1,es Spain,...,8.0,0.0,0.0,0.00,NaN,22.3,2.0,2.0,2019,United States Women
14,2019-06-28,21:00,World Cup,Quarter-finals,Fri,Away,W,2,1,fr France,...,10.0,8.0,80.0,0.20,0.25,18.9,0.0,0.0,2019,United States Women
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2015-06-11,19:00,World Cup,Group stage,Thu,Neutral,L,2,3,th Thailand,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2015,Cote dIvoire Women
2,2015-06-15,17:00,World Cup,Group stage,Mon,Neutral,L,1,3,no Norway,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2015,Cote dIvoire Women
0,2015-06-08,16:00,World Cup,Group stage,Mon,Neutral,L,0,6,cm Cameroon,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2015,Ecuador Women
1,2015-06-12,16:00,World Cup,Group stage,Fri,Neutral,L,1,10,ch Switzerland,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2015,Ecuador Women


In [20]:
WWC_past_matches.to_csv("WWC_past_matches.csv")